### Creating Bedrock Agents

Using AWS Bedrock and AWS Lambda

✅ Create an Amazon Bedrock agent

✅ Create an AWS Lambda function to access and modify sample employee data

✅ Create an action group for the Amazon Bedrock agent

✅ Test the agent with different prompts

In [1]:
import json
import time
import zipfile
from io import BytesIO
import uuid
import pprint
import logging
import boto3

print(boto3.__version__)

1.42.27


In [2]:
logger = logging.getLogger(__name__)

In [3]:
sts_client = boto3.client('sts')
iam_client = boto3.client('iam')
lambda_client = boto3.client('lambda')
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')

In [4]:
session = boto3.session.Session()
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]

region, account_id

('eu-west-2', '035663780217')

In [5]:
inference_profile = "us.amazon.nova-micro-1:0"
foundation_model = inference_profile[3:]

foundation_model

'amazon.nova-micro-1:0'

In [7]:
suffix = f"{region}-{account_id}"
agent_name = "hr-assistant-function-def"
agent_bedrock_allow_policy_name = f"{agent_name}-ba-{suffix}"
agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{agent_name}'
agent_description = "Agent for providing HR assistance to manage vacation time"
agent_instruction = "You are an HR agent, helping employees understand HR policies and manage vacation time"
agent_action_group_name = "VacationsActionGroup"
agent_action_group_description = "Actions for getting the number of available vacations days for an employee and confirm new time off"
agent_alias_name = f"{agent_name}-alias"
lambda_function_role = f'{agent_name}-lambda-role-{suffix}'
lambda_function_name = f'{agent_name}-{suffix}'

lambda_function_name

'hr-assistant-function-def-eu-west-2-035663780217'

In [11]:
# creating employee database to be used by lambda function
import sqlite3
import random
from datetime import date, timedelta

# Connect to the SQLite database (creates a new one if it doesn't exist)
conn = sqlite3.connect('employee_database.db')
c = conn.cursor()

# Create the employees table
c.execute('''CREATE TABLE IF NOT EXISTS employees
                (employee_id INTEGER PRIMARY KEY AUTOINCREMENT, employee_name TEXT, employee_job_title TEXT, employee_start_date TEXT, employee_employment_status TEXT)''')

# Create the vacations table
c.execute('''CREATE TABLE IF NOT EXISTS vacations
                (employee_id INTEGER, year INTEGER, employee_total_vacation_days INTEGER, employee_vacation_days_taken INTEGER, employee_vacation_days_available INTEGER, FOREIGN KEY(employee_id) REFERENCES employees(employee_id))''')

# Create the planned_vacations table
c.execute('''CREATE TABLE IF NOT EXISTS planned_vacations
                (employee_id INTEGER, vacation_start_date TEXT, vacation_end_date TEXT, vacation_days_taken INTEGER, FOREIGN KEY(employee_id) REFERENCES employees(employee_id))''')

# Generate some random data for 10 employees
employee_names = ['John Doe', 'Jane Smith', 'Bob Johnson', 'Alice Williams', 'Tom Brown', 'Emily Davis', 'Michael Wilson', 'Sarah Taylor', 'David Anderson', 'Jessica Thompson']
job_titles = ['Manager', 'Developer', 'Designer', 'Analyst', 'Accountant', 'Sales Representative']
employment_statuses = ['Active', 'Inactive']

for i in range(10):
    name = employee_names[i]
    job_title = random.choice(job_titles)
    start_date = date(2015 + random.randint(0, 7), random.randint(1, 12), random.randint(1, 28)).strftime('%Y−%m−%d')
    employment_status = random.choice(employment_statuses)
    c.execute("INSERT INTO employees (employee_name, employee_job_title, employee_start_date, employee_employment_status) VALUES (?, ?, ?, ?)", (name, job_title, start_date, employment_status))
    employee_id = c.lastrowid

    # Generate vacation data for the current employee
    for year in range(date.today().year, date.today().year - 3, -1):
        total_vacation_days = random.randint(10, 30)
        days_taken = random.randint(0, total_vacation_days)
        days_available = total_vacation_days - days_taken
        c.execute("INSERT INTO vacations (employee_id, year, employee_total_vacation_days, employee_vacation_days_taken, employee_vacation_days_available) VALUES (?, ?, ?, ?, ?)", (employee_id, year, total_vacation_days, days_taken, days_available))

        # Generate some planned vacations for the current employee and year
        num_planned_vacations = random.randint(0, 3)
        for _ in range(num_planned_vacations):
            start_date = date(year, random.randint(1, 12), random.randint(1, 28)).strftime('%Y−%m−%d')
            end_date = (date(int(start_date[:4]), int(start_date[5:7]), int(start_date[8:])) + timedelta(days=random.randint(1, 14))).strftime('%Y−%m−%d')
            days_taken = (date(int(end_date[:4]), int(end_date[5:7]), int(end_date[8:])) - date(int(start_date[:4]), int(start_date[5:7]), int(start_date[8:])))
            c.execute("INSERT INTO planned_vacations (employee_id, vacation_start_date, vacation_end_date, vacation_days_taken) VALUES (?, ?, ?, ?)", (employee_id, start_date, end_date, days_taken.days))

# Commit the changes and close the connection
conn.commit()
conn.close()

In [12]:
try:
    assume_role_policy_document = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Principal": {
                    "Service": "lambda.amazonaws.com"
                },
                "Action": "sts:AssumeRole"
            }
        ]
    }

    assume_role_policy_document_json = json.dumps(assume_role_policy_document)

    lambda_iam_role = iam_client.create_role(
        RoleName=lambda_function_role,
        AssumeRolePolicyDocument=assume_role_policy_document_json
    )

    # Pause to make sure role is created
    time.sleep(10)
except:
    lambda_iam_role = iam_client.get_role(RoleName=lambda_function_role)

iam_client.attach_role_policy(
    RoleName=lambda_function_role,
    PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'
)

{'ResponseMetadata': {'RequestId': '48b607f1-28ea-4efe-bde2-ccd81648c5e9',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 20 Jan 2026 12:16:10 GMT',
   'x-amzn-requestid': '48b607f1-28ea-4efe-bde2-ccd81648c5e9',
   'content-type': 'text/xml',
   'content-length': '212'},
  'RetryAttempts': 0}}

In [18]:
import os

# Package up the lambda function code
s = BytesIO()
z = zipfile.ZipFile(s, 'w')
z.write(f"{os.getcwd()}/agent_lambda/function.py")
z.write(f"{os.getcwd()}/agent_lambda/employee_database.db")
z.close()
zip_content = s.getvalue()

# Create Lambda Function
lambda_function = lambda_client.create_function(
    FunctionName=lambda_function_name,
    Runtime='python3.12',
    Timeout=180,
    Role=lambda_iam_role['Role']['Arn'],
    Code={'ZipFile': zip_content},
    Handler='lambda_function.lambda_handler'
)

In [19]:
### Creating Agent IAM Policy

bedrock_agent_bedrock_allow_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AmazonBedrockAgentBedrockFoundationModelPolicy",
            "Effect": "Allow",
            "Action": "bedrock:InvokeModel",
            "Resource": [
                f"arn:aws:bedrock:*::foundation-model/{foundation_model}",
                f"arn:aws:bedrock:*:*:inference-profile/{inference_profile}"
            ]
        },
        {
            "Sid": "AmazonBedrockAgentBedrockGetInferenceProfile",
            "Effect": "Allow",
            "Action":  [
                "bedrock:GetInferenceProfile",
                "bedrock:ListInferenceProfiles",
                "bedrock:UseInferenceProfile"
            ],
            "Resource": [
                f"arn:aws:bedrock:*:*:inference-profile/{inference_profile}"
            ]
        }
    ]
}

bedrock_policy_json = json.dumps(bedrock_agent_bedrock_allow_policy_statement)

agent_bedrock_policy = iam_client.create_policy(
    PolicyName=agent_bedrock_allow_policy_name,
    PolicyDocument=bedrock_policy_json
)

In [20]:
# Create IAM Role for the agent and attach IAM policies
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [{
        "Effect": "Allow",
        "Principal": {
            "Service": "bedrock.amazonaws.com"
        },
        "Action": "sts:AssumeRole"
    }]
}

assume_role_policy_document_json = json.dumps(assume_role_policy_document)
agent_role = iam_client.create_role(
    RoleName=agent_role_name,
    AssumeRolePolicyDocument=assume_role_policy_document_json
)

# Pause to make sure role is created
time.sleep(10)

iam_client.attach_role_policy(
    RoleName=agent_role_name,
    PolicyArn=agent_bedrock_policy['Policy']['Arn']
)

{'ResponseMetadata': {'RequestId': '94cea386-90cd-495d-ac52-8307d6e0a62d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 20 Jan 2026 12:21:54 GMT',
   'x-amzn-requestid': '94cea386-90cd-495d-ac52-8307d6e0a62d',
   'content-type': 'text/xml',
   'content-length': '212'},
  'RetryAttempts': 0}}

In [21]:
### Create Agent

response = bedrock_agent_client.create_agent(
    agentName=agent_name,
    agentResourceRoleArn=agent_role['Role']['Arn'],
    description=agent_description,
    idleSessionTTLInSeconds=1800,
    foundationModel=inference_profile,
    instruction=agent_instruction,
)
agent_id = response['agent']['agentId']
agent_id, response

('GF9WF2UDQ4',
 {'ResponseMetadata': {'RequestId': '39b6069c-2cdf-46d6-9949-2a2339ef9fdc',
   'HTTPStatusCode': 202,
   'HTTPHeaders': {'date': 'Tue, 20 Jan 2026 12:22:29 GMT',
    'content-type': 'application/json',
    'content-length': '691',
    'connection': 'keep-alive',
    'x-amzn-requestid': '39b6069c-2cdf-46d6-9949-2a2339ef9fdc',
    'x-amz-apigw-id': 'Xe78zEJPLPEEsHQ=',
    'x-amzn-trace-id': 'Root=1-696f7384-4167ab613e15ee954b71ad5b'},
   'RetryAttempts': 0},
  'agent': {'agentId': 'GF9WF2UDQ4',
   'agentName': 'hr-assistant-function-def',
   'agentArn': 'arn:aws:bedrock:eu-west-2:035663780217:agent/GF9WF2UDQ4',
   'instruction': 'You are an HR agent, helping employees understand HR policies and manage vacation time',
   'agentStatus': 'CREATING',
   'foundationModel': 'us.amazon.nova-micro-1:0',
   'description': 'Agent for providing HR assistance to manage vacation time',
   'orchestrationType': 'DEFAULT',
   'idleSessionTTLInSeconds': 1800,
   'agentResourceRoleArn': 'ar

In [22]:
# Creat action group to associae lambda as tool for agent
agent_functions = [
    {
        'name': 'get_available_vacations_days',
        'description': 'get the number of vacations available for a certain employee',
        'parameters': {
            "employee_id": {
                "description": "the id of the employee to get the available vacations",
                "required": True,
                "type": "integer"
            }
        }
    },
    {
        'name': 'reserve_vacation_time',
        'description': 'reserve vacation time for a specific employee - you need all parameters to reserve vacation time',
        'parameters': {
            "employee_id": {
                "description": "the id of the employee for which time off will be reserved",
                "required": True,
                "type": "integer"
            },
            "start_date": {
                "description": "the start date for the vacation time",
                "required": True,
                "type": "string"
            },
            "end_date": {
                "description": "the end date for the vacation time",
                "required": True,
                "type": "string"
            }
        }
    },
]


In [23]:
# Now, we can configure and create an action group here:
agent_action_group_response = bedrock_agent_client.create_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupExecutor={
        'lambda': lambda_function['FunctionArn']
    },
    actionGroupName=agent_action_group_name,
    functionSchema={
        'functions': agent_functions
    },
    description=agent_action_group_description
)

agent_action_group_response

{'ResponseMetadata': {'RequestId': 'a9ed7177-5852-449e-b8c7-1b6a58b0e830',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 20 Jan 2026 12:25:29 GMT',
   'content-type': 'application/json',
   'content-length': '1335',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'a9ed7177-5852-449e-b8c7-1b6a58b0e830',
   'x-amz-apigw-id': 'Xe8ZEGZhLPEEHmQ=',
   'x-amzn-trace-id': 'Root=1-696f7439-140d094474e272cc778ae42a'},
  'RetryAttempts': 0},
 'agentActionGroup': {'agentId': 'GF9WF2UDQ4',
  'agentVersion': 'DRAFT',
  'actionGroupId': '1YYC1EEZEK',
  'actionGroupName': 'VacationsActionGroup',
  'description': 'Actions for getting the number of available vacations days for an employee and confirm new time off',
  'createdAt': datetime.datetime(2026, 1, 20, 12, 25, 29, 691199, tzinfo=tzutc()),
  'updatedAt': datetime.datetime(2026, 1, 20, 12, 25, 29, 691199, tzinfo=tzutc()),
  'actionGroupExecutor': {'lambda': 'arn:aws:lambda:eu-west-2:035663780217:function:hr-assistant-function-def-

In [24]:
response = lambda_client.add_permission(
    FunctionName=lambda_function_name,
    StatementId='allow_bedrock',
    Action='lambda:InvokeFunction',
    Principal='bedrock.amazonaws.com',
    SourceArn=f"arn:aws:bedrock:{region}:{account_id}:agent/{agent_id}",
)

In [25]:
# Creates DRAFT version of agent for internal testing

response = bedrock_agent_client.prepare_agent(
    agentId=agent_id
)
response

{'ResponseMetadata': {'RequestId': '83d5eb95-0819-4740-8b00-8b9b8ed66825',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Tue, 20 Jan 2026 12:26:33 GMT',
   'content-type': 'application/json',
   'content-length': '119',
   'connection': 'keep-alive',
   'x-amzn-requestid': '83d5eb95-0819-4740-8b00-8b9b8ed66825',
   'x-amz-apigw-id': 'Xe8jDEhnLPEEH-Q=',
   'x-amzn-trace-id': 'Root=1-696f7479-5026b3563188e41a63256bee'},
  'RetryAttempts': 0},
 'agentId': 'GF9WF2UDQ4',
 'agentStatus': 'PREPARING',
 'agentVersion': 'DRAFT',
 'preparedAt': datetime.datetime(2026, 1, 20, 12, 26, 33, 742077, tzinfo=tzutc())}

In [26]:
agent_id = response['agentId']

response = bedrock_agent_client.create_agent_alias(
    agentAliasName='test-alias-1',
    agentId=agent_id
)

response

{'ResponseMetadata': {'RequestId': '7d34a09a-2f45-4eeb-81f4-25e42fc940af',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Tue, 20 Jan 2026 12:27:35 GMT',
   'content-type': 'application/json',
   'content-length': '382',
   'connection': 'keep-alive',
   'x-amzn-requestid': '7d34a09a-2f45-4eeb-81f4-25e42fc940af',
   'x-amz-apigw-id': 'Xe8suGsvrPEEprw=',
   'x-amzn-trace-id': 'Root=1-696f74b7-0d7bd82a61fbb2447c667097'},
  'RetryAttempts': 0},
 'agentAlias': {'agentId': 'GF9WF2UDQ4',
  'agentAliasId': 'CVUZHZGAJZ',
  'agentAliasName': 'test-alias-1',
  'agentAliasArn': 'arn:aws:bedrock:eu-west-2:035663780217:agent-alias/GF9WF2UDQ4/CVUZHZGAJZ',
  'routingConfiguration': [{}],
  'createdAt': datetime.datetime(2026, 1, 20, 12, 27, 35, 526098, tzinfo=tzutc()),
  'updatedAt': datetime.datetime(2026, 1, 20, 12, 27, 35, 526098, tzinfo=tzutc()),
  'agentAliasStatus': 'CREATING',
  'aliasInvocationState': 'ACCEPT_INVOCATIONS'}}

In [28]:
agent_alias_id = response["agentAlias"]["agentAliasId"]

agent_alias_id

'CVUZHZGAJZ'

In [ ]:
### Invoke agent

## create a random id for session initiator id
session_id:str = str(uuid.uuid1())
enable_trace:bool = False
end_session:bool = False

# invoke the agent API
agentResponse = bedrock_agent_runtime_client.invoke_agent(
    inputText="How much vacation does the employee with employee_id set to 1 have available?",
    agentId=agent_id,
    agentAliasId=agent_alias_id, 
    sessionId=session_id,
    enableTrace=enable_trace, 
    endSession= end_session
)

logger.info(pprint.pprint(agentResponse))

event_stream = agentResponse['completion']
try:
    for event in event_stream:        
        if 'chunk' in event:
            data = event['chunk']['bytes']
            logger.info(f"Final answer ->\n{data.decode('utf8')}")
            agent_answer = data.decode('utf8')
            end_event_received = True
        elif 'trace' in event:
            logger.info(json.dumps(event['trace'], indent=2))
        else:
            raise Exception("unexpected event.", event)
except Exception as e:
    raise Exception("unexpected event.", e)

In [ ]:
agentResponse = bedrock_agent_runtime_client.invoke_agent(
    inputText="Great. please reserve one day of time off for the employee with employee_id set to 1 for <FILL_ME_IN>",
    agentId=agent_id,
    agentAliasId=agent_alias_id, 
    sessionId=session_id,
    enableTrace=enable_trace, 
    endSession= end_session
)

logger.info(pprint.pprint(agentResponse))

event_stream = agentResponse['completion']
try:
    for event in event_stream:        
        if 'chunk' in event:
            data = event['chunk']['bytes']
            logger.info(f"Final answer ->\n{data.decode('utf8')}")
            agent_answer = data.decode('utf8')
            end_event_received = True
            # End event indicates that the request finished successfully
        elif 'trace' in event:
            logger.info(json.dumps(event['trace'], indent=2))
        else:
            raise Exception("unexpected event.", event)
except Exception as e:
    raise Exception("unexpected event.", e)

In [ ]:
agentResponse = bedrock_agent_runtime_client.invoke_agent(
    inputText="How much vacation does the employee with employee_id set to 1 have available?",
    agentId=agent_id,
    agentAliasId=agent_alias_id, 
    sessionId=session_id,
    enableTrace=enable_trace, 
    endSession= end_session
)

logger.info(pprint.pprint(agentResponse))

event_stream = agentResponse['completion']
try:
    for event in event_stream:        
        if 'chunk' in event:
            data = event['chunk']['bytes']
            logger.info(f"Final answer ->\n{data.decode('utf8')}")
            agent_answer = data.decode('utf8')
            end_event_received = True
        elif 'trace' in event:
            logger.info(json.dumps(event['trace'], indent=2))
        else:
            raise Exception("unexpected event.", event)
except Exception as e:
    raise Exception("unexpected event.", e)

In [30]:
action_group_id = agent_action_group_response['agentActionGroup']['actionGroupId']
action_group_name = agent_action_group_response['agentActionGroup']['actionGroupName']

response = bedrock_agent_client.update_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupId= action_group_id,
    actionGroupName=action_group_name,
    actionGroupExecutor={
        'lambda': lambda_function['FunctionArn']
    },
    functionSchema={
        'functions': agent_functions
    },
    actionGroupState='DISABLED',
)

action_group_deletion = bedrock_agent_client.delete_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupId= action_group_id
)

In [31]:
response = bedrock_agent_client.delete_agent_alias(
    agentAliasId=agent_alias_id,
    agentId=agent_id
)

In [32]:
agent_deletion = bedrock_agent_client.delete_agent(
    agentId=agent_id
)

In [33]:
lambda_client.delete_function(
    FunctionName=lambda_function_name
)

{'ResponseMetadata': {'RequestId': 'd7b340e2-fe24-4d05-b681-9fce6aaabc0b',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'date': 'Tue, 20 Jan 2026 12:35:40 GMT',
   'content-type': 'application/json',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'd7b340e2-fe24-4d05-b681-9fce6aaabc0b'},
  'RetryAttempts': 1},
 'StatusCode': 204}